## Cats

In [ ]:
import $ivy.`org.typelevel::cats-core:0.9.0`

In [ ]:
import cats.implicits._
import cats.Semigroup
import cats.Monoid

### Semigroup
A semigroup for some given type ``A`` has a single operation (which we will call ``combine``), which takes two values of type ``A``, and returns a value of type ``A``. This operation must be guaranteed to be associative. That is to say that:
```
((a combine b) combine c)
```
must be the same as
```
(a combine (b combine c))
```

In [ ]:
Semigroup[Int].combine(1,2)

In [ ]:
Semigroup[Int].combineN(2,5)

In [ ]:
Semigroup[Int].combineAllOption(Seq(2,5,7,6))

In [ ]:
Semigroup[Int ⇒ Int].combine({ (x: Int) ⇒ x + 1 }, { (x: Int) ⇒ x * 10 }).apply(6)

In [ ]:
Semigroup[Option[Int]].combine(Option(1), Option(2))

In [ ]:
Semigroup[Option[Int]].combine(Option(1), None)

In [ ]:
Map("foo" -> Map("bar" -> 5)).combine(Map("foo" -> Map("bar" -> 6), "baz" -> Map()))

### Monoid
``Monoid`` extends the ``Semigroup`` type class, adding an empty method to semigroup's combine. The empty method must return a value that when combined with any other instance of that type returns the other instance, i.e.:
```
(combine(x, empty) == combine(empty, x) == x)
```

In [ ]:
Monoid[Int].combine(1, 2)

In [ ]:
Monoid[Int].empty
Monoid[Float].empty
Monoid[String].empty

In [ ]:
Monoid[(Int, Double)].empty
Monoid[(Int, Double)].combine( (1, 5.2), (2, 2.3) )

In [ ]:
identity(1,2,3,4,5,6,7,8,9,10)
identity(Seq("a", "b", "c"))
identity(Seq(5, "a", 2.0, 'C'))

In [ ]:
Seq(5, "a", 2.0, 'C').map(identity)
Seq(5, "a", 2.0, 'C').toList.map(identity)
List(5, "a", 2.0, 'C').foldMap(identity) // method extension from Monoid

In [ ]:
List(6.02, "x", 10, 'e', '+', 23).foldMap(item => item.toString)

## Functor
A ``Functor`` is a ubiquitous type class involving types that have one "hole", i.e. types which have the shape ``F[?]``, such as ``Option``, ``List`` and ``Future``. (This is in contrast to a type like ``Int`` which has no hole, or ``Tuple2`` which has two holes ``(Tuple2[?,?])``).

The ``Functor`` category involves a single operation, named ``map``:
```
def map[A, B](fa: F[A])(f: A => B): F[B]
```

### Apply
``Apply`` extends the ``Functor`` type class (which features the familiar map function) with a new function ``ap``. The ``ap`` function is similar to ``map`` in that we are transforming a value in a context (a context being the ``F`` in ``F[A]``; a context can be ``Option``, ``List`` or ``Future`` for example). However, the difference between ``ap`` and ``map`` is that for ``ap`` the function that takes care of the transformation is of type ``F[A => B]``, whereas for ``map`` it is ``A => B``:
```
import cats._

implicit val optionApply: Apply[Option] = new Apply[Option] {
  def ap[A, B](f: Option[A => B])(fa: Option[A]): Option[B] =
    fa.flatMap(a => f.map(ff => ff(a)))

  def map[A, B](fa: Option[A])(f: A => B): Option[B] = fa map f

  def product[A, B](fa: Option[A], fb: Option[B]): Option[(A, B)] =
    fa.flatMap(a => fb.map(b => (a, b)))
}
```


### Aplicative
Applicative extends Apply by adding a single method, ``pure``:
```
def pure[A](x: A): F[A]
```

### Monad
``Monad`` extends the ``Applicative`` type class with a new function ``flatten`` which takes a value in a nested context (eg. ``F[F[A]]`` where ``F`` is the context) and "joins" the contexts together so that we have a single context (i.e.: ``F[A]``).

The name ``flatten`` should remind you of the functions of the same name on many classes in the standard library.

### Foldable
``Foldable`` type class instances can be defined for data structures that can be folded to a summary value.

In the case of a collection (such as ``List`` or ``Set``), these methods will fold together (``combine``) the values contained in the collection to produce a single result. Most collection types have ``foldLeft`` methods, which will usually be used by the associated ``Foldable[_]`` instance.

``Foldable[F]`` is implemented in terms of two basic methods:

* ``foldLeft(fa, b)(f)`` eagerly folds fa from left-to-right.
* ``foldRight(fa, b)(f)`` lazily folds fa from right-to-left.

### Traverse
At center stage of ``Traverse`` is the ``traverse`` method:
```
trait Traverse[F[_]] {
  def traverse[G[_]: Applicative, A, B](fa: F[A])(f: A => G[B]): G[F[B]]
}
```
For example, ``F`` is ``List``, and ``G`` is ``Option``, ``Xor``, or ``Future``. Now, for a given ``List[User]`` and a function ``User => Future[Profile]``, ``traverse`` can give you a ``Future[List[Profile]]``.
